In [1]:
import pandas as pd
df = pd.read_csv('Espacenet_search_result.csv', delimiter=';')

In [2]:
df[['first publication date','second publication date']] = df['Publication date'].str.split(' ' , n=1 , expand= True)
df['second publication date'] = df['second publication date'].str.strip('\n')
df.head()

,No,Title,Inventors,Applicants,Publication number,Earliest priority,IPC,CPC,Publication date,Earliest publication,Family number,Unnamed: 11,first publication date,second publication date
0,1,Plug-in hybrid vehicle with fast energy storage,BENDER DONALD A [US] \nDESHMANE ATUL [US] \nME...,AFS TRINITY POWER CORP [US],US2006250902A1,2005-05-05,H04B1/20,"B60K6/20 (KR) \nB60K6/28 (KR) \nB60K6/30 (EP,U...",2006-11-09,2006-11-09,37397092,NaN,2006-11-09,None
1,2,Electric vehicle wireless charging system thro...,SON JEONG KI [KR],SON JEONG KI [KR],KR20230163874A,2022-05-24,B60L53/12 \nB60L53/38 \nB60L53/54 \nB60L53/66 ...,B60L53/12 (KR) \nB60L53/38 (KR) \nB60L53/54 (K...,2023-12-01,2023-12-01,89124565,NaN,2023-12-01,None
2,3,Electric vehicle charging system using hydroge...,HONG SEONG HO,HOGREENAIR CO LTD [KR],KR102511391B1,2022-09-20,B60L3/00 \nB60L53/16 \nB60L53/51 \nB60L53/54 \...,Y02T10/70 (EP),2023-03-20,2023-03-20,85796535,NaN,2023-03-20,None
3,4,BATTERY-CHARGING SYSTEM FOR AN ELECTRIC VEHICLE,CHUNG YON JONG [KR],CHUNG YON JONG [KR],WO2011019133A2 \nWO2011019133A3,2009-08-13,B60L11/18 \nB60W10/24 \nH02J7/00,B60K6/46 (EP) \nB60L15/2045 (EP) \nB60L58/12 (...,2011-02-17 \n2011-04-07,2010-03-25,42183715,NaN,2011-02-17,2011-04-07
4,5,ESS SYSTEM FOR CHARGING FUEL CELL ELECTRIC VEH...,MUN TAEEUN [KR] \nCHAE HO BYUNG [KR] \nSHIN JO...,SIGNET EV INC [KR] \nSIGNET ENERGY [KR],US2022140365A1,2020-11-02,H01M16/00 \nH01M8/04828 \nH02J7/34,B60K15/03006 (KR) \nB60L53/20 (KR) \nB60L53/50...,2022-05-05,2021-03-17,75243546,NaN,2022-05-05,None


In [3]:
def click_more_options_button(self):
    """Find and click the 'More Options' button."""
    try:
        # Debug: Print all buttons on the page
        buttons = self.driver.find_elements(By.TAG_NAME, "button")
        print(f"Found {len(buttons)} buttons on the page:")
        for button in buttons:
            print(f"Button text: {button.text}, Class: {button.get_attribute('class')}")

        # Wait for the "More Options" button to be clickable
        more_options_button = WebDriverWait(self.driver, 20).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, "button.prod-jss36.prod-jss27"))
        )

        # Click the "More Options" button
        more_options_button.click()
        print("Clicked 'More Options' button")

        # Debug: Print all dropdown menus on the page
        dropdown_menus = self.driver.find_elements(By.CSS_SELECTOR, "div.prod-jss546")
        print(f"Found {len(dropdown_menus)} dropdown menus on the page:")
        for menu in dropdown_menus:
            print(f"Dropdown menu HTML: {menu.get_attribute('innerHTML')}")

        # Wait for the dropdown menu to appear
        WebDriverWait(self.driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "div.prod-jss546"))
        )

    except Exception as e:
        print(f"Error clicking 'More Options' button: {e}")

def click_download_button(self):
    """Find and click the 'Download' button in the dropdown menu."""
    try:
        # Debug: Print all download buttons on the page
        download_buttons = self.driver.find_elements(By.CSS_SELECTOR, "section.prod-jss36")
        print(f"Found {len(download_buttons)} download buttons on the page:")
        for button in download_buttons:
            print(f"Download button text: {button.text}, Class: {button.get_attribute('class')}")

        # Wait for the "Download" button to be clickable
        download_button = WebDriverWait(self.driver, 20).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, "section.prod-jss36.prod-jss700"))
        )

        # Click the "Download" button
        download_button.click()
        print("Clicked 'Download' button")

        # Wait for the file to be downloaded
        time.sleep(10)  # Adjust this delay based on your network speed

        # Find the downloaded file
        downloaded_file = os.path.join(self.download_dir, "search_results.csv")
        if os.path.exists(downloaded_file):
            print(f"File downloaded successfully: {downloaded_file}")

            # Read the CSV file
            with open(downloaded_file, mode='r', encoding='utf-8') as file:
                csv_reader = csv.DictReader(file)
                for row in csv_reader:
                    print(row)
        else:
            print("File not found. Check the download directory.")

    except Exception as e:
        print(f"Error clicking 'Download' button: {e}")

In [4]:
def click_more_options_button(self):
    """Find and click the 'Menu' button."""
    try:
        # Wait for the "Menu" button to be clickable
        more_options_button = WebDriverWait(self.driver, 20).until(
            EC.element_to_be_clickable((By.XPATH, "//button[contains(@aria-label, 'Menu') or contains(text(), 'Menu')]"))
        )

        # Click the "Menu" button
        more_options_button.click()
        print("Clicked 'Menu' button")

        # Add a delay to wait for the dropdown to appear
        time.sleep(2)  # Adjust this delay as needed

        # Debug: Print the HTML of the dropdown menu
        dropdown_menus = self.driver.find_elements(By.CSS_SELECTOR, "div.prod-jss546")
        print(f"Found {len(dropdown_menus)} dropdown menus on the page:")
        for menu in dropdown_menus:
            print(f"Dropdown menu HTML: {menu.get_attribute('innerHTML')}")

        # Wait for the dropdown menu to appear
        WebDriverWait(self.driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "div.prod-jss546"))
        )

    except Exception as e:
        print(f"Error clicking 'Menu' button: {e}")

def click_download_button(self):
    """Find and click the 'Télécharger' button in the dropdown menu."""
    try:
        # Wait for the "Télécharger" button to be clickable
        download_button = WebDriverWait(self.driver, 20).until(
            EC.element_to_be_clickable((By.XPATH, "//section[contains(text(), 'Télécharger')]"))
        )

        # Click the "Télécharger" button
        download_button.click()
        print("Clicked 'Télécharger' button")

        # Wait for the file to be downloaded
        time.sleep(10)  # Adjust this delay based on your network speed

        # Find the downloaded file
        downloaded_file = os.path.join(self.download_dir, "search_results.csv")
        if os.path.exists(downloaded_file):
            print(f"File downloaded successfully: {downloaded_file}")

            # Read the CSV file
            with open(downloaded_file, mode='r', encoding='utf-8') as file:
                csv_reader = csv.DictReader(file)
                for row in csv_reader:
                    print(row)
        else:
            print("File not found. Check the download directory.")

    except Exception as e:
        print(f"Error clicking 'Télécharger' button: {e}")

In [ ]:
#replacing the family number column 
import time
import random
import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from bs4 import BeautifulSoup
import pandas as pd

class EspacenetScraper:
    def __init__(self, headless=True):
        """Initialize the scraper with configurable options."""
        options = uc.ChromeOptions()
        if headless:
            options.add_argument('--headless')  # Run in headless mode

        options.add_argument('--disable-blink-features=AutomationControlled')
        self.driver = uc.Chrome(options=options)
        self.driver.set_page_load_timeout(30)
        self.driver.set_window_size(1920, 1080)

    def add_random_delay(self, min_seconds=1, max_seconds=3):
        """Add a random delay to mimic human behavior."""
        time.sleep(random.uniform(min_seconds, max_seconds))

    def get_page_html(self, url):
        """Navigate to the given URL and return the page HTML."""
        try:
            print(f"Navigating to: {url}")
            self.driver.get(url)

            # Wait for the page to load completely
            WebDriverWait(self.driver, 20).until(
                EC.presence_of_element_located((By.TAG_NAME, "body"))
            )

            # Add a random delay to mimic human behavior
            self.add_random_delay(3, 5)

            # Return the page HTML
            return self.driver.page_source

        except TimeoutException:
            print("Timed out waiting for the page to load.")
            return None
        except Exception as e:
            print(f"An error occurred: {e}")
            return None

    def parse_html(self, html):
        """Parse the HTML and extract all span elements inside the 'Published as' content."""
        soup = BeautifulSoup(html, 'html.parser')
        
        # Look for the element containing "Publié en tant que" or "Published as"
        published_as_element = soup.find(lambda tag: tag.name == "h5" and ("Publié en tant que" in tag.text or "Published as" in tag.text))
        
        if published_as_element:
            # Get the next sibling span that contains the relevant content
            content_element = published_as_element.find_next_sibling("span")
            if content_element:
                # Extract all span elements within the content
                spans = content_element.find_all('span')
                return [span.get_text(strip=True) for span in spans]
        return []

    def close(self):
        """Close the browser when done."""
        if self.driver:
            self.driver.quit()

# Example Usage:
if __name__ == '__main__':
    # Initialize the scraper
    scraper = EspacenetScraper(headless=False)  # Set headless to False to see the browser in action

    # Create a DataFrame with family numbers and publication numbers
    #data = {
     #   'family_number': ['087517563', 'another_family_number'],  # Replace with actual family numbers
      #  'publication_number': ['GB2631304A', 'another_publication_number']  # Replace with actual publication numbers
  #  }
   # df = pd.DataFrame(data)

    # Create a new column for family members
    df['family_members'] = None

    try:
        for index, row in df.iterrows():
            # Construct the URL using family number and publication number
            url = f"https://worldwide.espacenet.com/patent/search/family/{row['Family number']}/publication/{row['first publication number']}?q=hydrogen%20battery"

            # Get the page HTML
            html = scraper.get_page_html(url)
            if html:
                print(f"Page HTML retrieved successfully for {row['first publication number']}.")
                # Parse the HTML to find the relevant content
                family_members = scraper.parse_html(html)
                df.at[index, 'family_members'] = family_members  # Store the result in the DataFrame
            else:
                print(f"Failed to retrieve the page HTML for {row['first publication number']}.")

    finally:
        # Close the browser
        scraper.close()
        print("Scraper closed.")

    # Display the DataFrame with
    df.head()